In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#scraping URL where release is uploaded weekly
my_url = "https://www.dol.gov/newsroom/releases/"
raw_html = requests.get(my_url).content

In [3]:
soup_doc = BeautifulSoup(raw_html, "html.parser")
print(soup_doc.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <link href="https://www.dol.gov/newsroom/releases" rel="canonical"/>
  <meta content="DOL" property="og:site_name"/>
  <meta content="http://www.dol.gov/newsroom/releases" property="og:url"/>
  <meta content="News Releases" property="og:title"/>
  <meta content="http://www.dol.gov/sites/dolgov/files/OPA/twitter-cards/HomepageTwittercardOmbre.png" property="og:image"/>
  <meta content="http://www.dol.gov/sites/dolgov/files/OPA/twitter-cards/HomepageTwittercardOmbre.png" property="og:image:url"/>
  <meta content="https://www.dol.gov/sites/dolgo

In [4]:
# Find the element containing the specified text
text = soup_doc.find('span', string="Unemployment Insurance Weekly Claims Report")
link = text.find_parent('a')['href'] 
print(link)

    /newsroom/releases/eta/eta20231228



In [5]:
base_url = "https://www.dol.gov"
pdf_url = base_url + link.strip()
print (pdf_url)

https://www.dol.gov/newsroom/releases/eta/eta20231228


In [6]:
# Scraping PDFs
import camelot

In [7]:
tables = camelot.read_pdf(pdf_url, flavor='stream', pages='1-end')
len(tables)

/Users/radhikarukmangadhan/.pyenv/versions/3.11.6/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


10

In [8]:
# Find table with relevant info
df = tables[5].df
df

,0,1,2,3,4,5,6
0,,,Advance State Claims - Not Seasonally Adjusted,,,,
1,,,Initial Claims Filed During Week Ended Decembe...,,,Insured Unemployment For Week Ended December 16,
2,STATE,Advance,Prior Wk,Change,Advance,Prior Wk,Change
3,Alabama,"2,282","2,274",8,"8,674","9,376",-702
4,Alaska,879,798,81,"6,619","6,755",-136
5,Arizona,"2,943","3,264",-321,"20,512","21,661","-1,149"
6,Arkansas,"2,160","1,493",667,"9,709","10,330",-621
7,California **,"49,098","43,504","5,594","382,949","386,931","-3,982"
8,Colorado,"1,850","2,118",-268,"30,212","29,396",816
9,Connecticut,"6,123","3,859","2,264","25,931","23,529","2,402"


In [9]:
# Dropping unwanted rows and columns
df = df.drop(columns=[1,2,4, 5, 6])
df = df.drop([0, 1, 2])
df

,0,3
3,Alabama,8
4,Alaska,81
5,Arizona,-321
6,Arkansas,667
7,California **,"5,594"
8,Colorado,-268
9,Connecticut,"2,264"
10,Delaware,156
11,District of Columbia,-46
12,Florida,-506


In [10]:
# Naming columns
df.columns=["State","Change"]
df = df.replace({',': '','\*': ''}, regex=True)
df

,State,Change
3,Alabama,8
4,Alaska,81
5,Arizona,-321
6,Arkansas,667
7,California,5594
8,Colorado,-268
9,Connecticut,2264
10,Delaware,156
11,District of Columbia,-46
12,Florida,-506


In [11]:
# Downloading CSV in append mode
df.to_csv("statewise_claims.csv", index = False)